In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('..')

from backtest.performance_generater import PerformanceGenerator
from backtest.operators import *

In [2]:
open = pd.read_csv('/Volumes/Database/Factor_data/open.csv', index_col=0, parse_dates=True)
Close = pd.read_csv('/Volumes/Database/Factor_data/close.csv', index_col=0, parse_dates=True)
High = pd.read_csv('/Volumes/Database/Factor_data/high.csv', index_col=0, parse_dates=True)
Low = pd.read_csv('/Volumes/Database/Factor_data/low.csv', index_col=0, parse_dates=True)
Volume = pd.read_csv('/Volumes/Database/Factor_data/volume.csv', index_col=0, parse_dates=True)
returns = pd.read_csv('/Volumes/Database/Factor_data/return.csv', index_col=0, parse_dates=True)
exp_returns = returns.shift(-2) * 0.01

# operatingmargin = pd.read_csv('/Volumes/Database/Factor_data/營業利益率_90.csv', index_col=0, parse_dates=True) * 0.01
# pb_q = pd.read_csv('/Volumes/Database/Factor_data/當季季底PB_90.csv', index_col=0, parse_dates=True)
revenue_m = pd.read_csv('/Volumes/Database/Factor_data/以合併為主單月營收(千元)_90.csv', index_col=0, parse_dates=True)
# operatingincome = pd.read_csv('/Volumes/Database/Factor_data/營業利益_90.csv', index_col=0, parse_dates=True)
# operatingincome = pd.read_csv('/Volumes/Database/Factor_data/單月營業利益(千元)_90.csv', index_col=0, parse_dates=True)
# outstanding = pd.read_csv('/Volumes/Database/Factor_data/流通在外股數(千股).csv', index_col=0, parse_dates=True)
# CR = pd.read_csv('/Volumes/Database/Factor_data/流動比率_90.csv', index_col=0, parse_dates=True)
# CD = pd.read_csv('/Volumes/Database/Factor_data/流動負債_90.csv', index_col=0, parse_dates=True)
CA = pd.read_csv('/Volumes/Database/Factor_data/流動資產_90.csv', index_col=0, parse_dates=True)
# inv = pd.read_csv('/Volumes/Database/Factor_data/  存貨_90.csv', index_col=0, parse_dates=True)
# totalD = pd.read_csv('/Volumes/Database/Factor_data/負債總額_90.csv', index_col=0, parse_dates=True)
# totalasset = pd.read_csv('/Volumes/Database/Factor_data/資產總額_90.csv', index_col=0, parse_dates=True)
# netasset = pd.read_csv('/Volumes/Database/Factor_data/淨值資產_90.csv', index_col=0, parse_dates=True)
# netdebt = pd.read_csv('/Volumes/Database/Factor_data/淨負債_90.csv', index_col=0, parse_dates=True)
# NI = pd.read_csv('/Volumes/Database/Factor_data/稀釋稅後淨利_90.csv', index_col=0, parse_dates=True)
# revenue_yoy = pd.read_csv('/Volumes/Database/Factor_data/單月營收成長率％_90.csv', index_col=0, parse_dates=True)
# DR = pd.read_csv('/Volumes/Database/Factor_data/負債比率_90.csv', index_col=0, parse_dates=True)
# FIIBV = pd.read_csv('/Volumes/Database/Factor_data/外資買進張數.csv', index_col=0, parse_dates=True)
# ITBV = pd.read_csv('/Volumes/Database/Factor_data/投信買進張數.csv', index_col=0, parse_dates=True)
# PTBV = pd.read_csv('/Volumes/Database/Factor_data/自營商買進張數.csv', index_col=0, parse_dates=True)
# FIISV = pd.read_csv('/Volumes/Database/Factor_data/外資賣出張數.csv', index_col=0, parse_dates=True)
# ITSV = pd.read_csv('/Volumes/Database/Factor_data/投信賣出張數.csv', index_col=0, parse_dates=True)
# PTSV = pd.read_csv('/Volumes/Database/Factor_data/自營商賣出張數.csv', index_col=0, parse_dates=True)

存貨週轉率（次）_90


In [3]:
ZTXA_close = pd.read_csv('/Volumes/Database/Factor_data/ZXTA_close.csv', index_col=0)
ZTXA_close.index = pd.to_datetime(ZTXA_close.index)
market_returns = ZTXA_close['ZTXA'].pct_change()
Benchmark = ZTXA_close['ZTXA'].pct_change().shift(-2)

In [4]:
# factor = rolling_regression(operatingincome/asset, returns, 315, 'alpha')

In [15]:
inv_turnover

,1101,1102,1103,1104,1107,1108,1109,1110,1113,1201,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-06,1.52,2.3,4.28,3.44,0.0,0.25,1.41,0.48,NaN,2.49,...,0.94,0.13,0.01,0.06,0.9,0.96,0.67,1.73,0.84,0.86
2024-03-07,1.52,2.3,4.28,3.44,0.0,0.25,1.41,0.48,NaN,2.49,...,0.94,0.13,0.01,0.06,0.9,0.96,0.67,1.73,0.84,0.86
2024-03-08,1.52,2.3,4.28,3.44,0.0,0.25,1.41,0.48,NaN,2.49,...,0.94,0.13,0.01,0.06,0.9,0.96,0.67,1.73,0.84,0.86


In [14]:
inv_turnover = pd.read_csv('/Volumes/Database/Factor_data/存貨週轉率（次）_90.csv', index_col=0, parse_dates=True)
factor = ts_rank(inv_turnover, 252)

# no fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     buy_fee = 0, 
                                                     sell_fee = 0,
                                                     benchmark = Benchmark
                                                     ).backtest()

# fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     benchmark = Benchmark
                                                     ).backtest()

/Users/tedting/Documents/FactorTradingSystem/test/../backtest/operators.py:71: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |        0.00 %         |        0.00 %        |     nan      |     0.00 %     |    0.00 %    |   0.00 %   | 0.00 % |  0.00 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


/Users/tedting/Documents/FactorTradingSystem/test/../backtest/performance_generater.py:90: RuntimeWarning:

invalid value encountered in scalar divide



+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |        0.00 %         |        0.00 %        |     nan      |     0.00 %     |    0.00 %    |   0.00 %   | 0.00 % |  0.00 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


/Users/tedting/Documents/FactorTradingSystem/test/../backtest/performance_generater.py:90: RuntimeWarning:

invalid value encountered in scalar divide



In [13]:
csv_file = '/Volumes/Database/medina_alpha/inv_turnover.csv'
factor.to_csv(csv_file)